## Setup

Execute the cell below to setup the notebook to run Custom diffusion

In [ ]:
!bash setup.sh 

# Training

## Diffusers method

In [ ]:
## launch training script (2 GPUs recommended, increase --max_train_steps to 500 if 1 GPU)

!accelerate launch src/diffuser_training.py \
          --pretrained_model_name_or_path=compvis/stable-diffusion-v1-4   \
          --instance_data_dir=./data/cat  \
          --class_data_dir=./real_reg/samples_cat/ \
          --output_dir=./logs/cat  \
          --with_prior_preservation --real_prior --prior_loss_weight=1.0 \
          --instance_prompt="photo of a <new1> cat"  \
          --class_prompt="cat" \
          --resolution=512  \
          --train_batch_size=2  \
          --learning_rate=1e-5  \
          --lr_warmup_steps=0 \
          --max_train_steps=250 \
          --num_class_images=200 \
          --scale_lr \
          --modifier_token "<new1>"

## sample 
python src/sample_diffuser.py --delta_ckpt logs/cat/delta.bin --ckpt "CompVis/stable-diffusion-v1-4" --prompt "<new1> cat playing with a ball"


In [ ]:
## launch training script (2 GPUs recommended, increase --max_train_steps to 1000 if 1 GPU)
## provide some json file with the info about each concept
!CUDA_VISIBLE_DEVICES=2,3 accelerate launch src/diffuser_training.py \
          --pretrained_model_name_or_path=compvis/stable-diffusion-v1-4  \
          --output_dir=./logs/cat_wooden_pot  \
          --concepts_list=./assets/concept_list.json \
          --with_prior_preservation --real_prior --prior_loss_weight=1.0 \
          --resolution=512  \
          --train_batch_size=2  \
          --learning_rate=1e-5  \
          --lr_warmup_steps=0 \
          --max_train_steps=500 \
          --num_class_images=200 \
          --scale_lr --hflip  \
          --modifier_token "<new1>+<new2>" 

## sample 
python src/sample_diffuser.py --delta_ckpt logs/cat_wooden_pot/delta.bin --ckpt "CompVis/stable-diffusion-v1-4" --prompt "<new1> cat sitting inside a <new2> wooden pot and looking up"

## Gradio application

In [ ]:
%cd ~/../custom-diffusion
!python app.py --share

# Sample from the newly trained model

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

%cd ~/../notebooks

## Set the args
device = 'cuda'
model_ = 'custom-diffusion/results/checkpoint-500'
size_ = 512
precision = 512
sample_num = 5

print(f'Generating samples from Stable Diffusion {model_} checkpoint ({precision})')


## Instantiate the model pipe with StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained(model_, torch_dtype=torch.float16, revision="fp16")
pipe = pipe.to(device)

for j in range(sample_num):
    a = pipe(prompt = 'a photo of a <new1> cat',
             negative_prompt = None,
             guidance_scale=7.5,
             height = 512,
             width = 512,
             num_images_per_prompt = 1,
             num_inference_steps=50)['images']    
    for i in a:
        display(i)
    # hash the next line out to save results
    #a.save(f'outputs/gen-image-{j}.png')


# Compress the model

In [ ]:
!python src/compress.py --delta_ckpt <finetuned-delta-path> --ckpt <pretrained-model-path>